<a href="https://colab.research.google.com/github/S3688570/Cars_Deep_Learning/blob/master/CNN_Cars_train_predict_final2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deep Learning Model to Recognize Vehicle Accidents

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
!ls "/content/drive/My Drive/"

'Colab Notebooks'  'Getting started.pdf'


In [0]:
!pip install -U -q PyDrive

In [0]:
# from google.colab import files
# uploaded = files.upload()

# New Section

##Import Keras library and packages

In [0]:
# Importing the Keras libraries and packages
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers import Convolution2D, MaxPooling2D, Dropout
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense
from keras import backend as K
from keras.preprocessing import image
from keras import regularizers
from PIL import Image
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import os

##Set parameters

In [0]:
epochs = 50 #@param {type:"number"}
validation_steps = 32 #@param {type:"number"}
img_height = 128 #@param {type:"integer"}
img_width = 128 #@param {type:"integer"}
batch_size = 32 #@param {type:"number"}


##Set data filepaths

In [0]:
# trainingCarImages = os.listdir('/content/drive/My Drive/Colab Notebooks/cars_dl/cars/training_set/car/')
# print ("Number of Training car images - ",str(len(trainingCarImages)))

# trainingCrashImages = os.listdir('/content/drive/My Drive/Colab Notebooks/cars_dl/cars/training_set/crash/')
# print ("Number of Training crash images - ",str(len(trainingCrashImages)))

# validCarImages = os.listdir('/content/drive/My Drive/Colab Notebooks/cars_dl/cars/val_set/car/')
# print ("Number of Validation car images - ",str(len(validCarImages)))

# validCrashImages = os.listdir('/content/drive/My Drive/Colab Notebooks/cars_dl/cars/val_set/crash/')
# print ("Number of Validation crash images - ",str(len(validCrashImages)))

# testCarImages = os.listdir('/content/drive/My Drive/Colab Notebooks/cars_dl/cars/test_set/car/')
# print ("Number of Test car images - ",str(len(testCarImages)))

# testCrashImages = os.listdir('/content/drive/My Drive/Colab Notebooks/cars_dl/cars/test_set/crash/')
# print ("Number of Test crash images - ",str(len(testCrashImages)))


trainingCarImages = os.listdir('cars/cars/training_set/car/')
print ("Number of Training car images - ",str(len(trainingCarImages)))

trainingCrashImages = os.listdir('cars/cars/training_set/crash/')
print ("Number of Training crash images - ",str(len(trainingCrashImages)))

validCarImages = os.listdir('cars/cars/val_set/car/')
print ("Number of Validation car images - ",str(len(validCarImages)))

validCrashImages = os.listdir('cars/cars/val_set/crash/')
print ("Number of Validation crash images - ",str(len(validCrashImages)))

testCarImages = os.listdir('cars/cars/test_set/car/')
print ("Number of Test car images - ",str(len(testCarImages)))

testCrashImages = os.listdir('cars/cars/test_set/crash/')
print ("Number of Test crash images - ",str(len(testCrashImages)))

FileNotFoundError: ignored

##Print image from training folder

In [0]:
trainFilename = '/content/drive/My Drive/Colab Notebooks/cars_dl/cars/training_set/car/'+trainingCarImages[2]
dimage = Image.open(trainFilename)
dimage

##Print image from validation folder

In [0]:
validFilename = '/content/drive/My Drive/Colab Notebooks/cars_dl/cars/val_set/car/'+validCarImages[5]
dimage = Image.open(validFilename)
dimage

##Print image from test folder

In [0]:
testFilename = '/content/drive/My Drive/Colab Notebooks/cars_dl/cars/test_set/car/'+ testCarImages[30]
dimage = Image.open(testFilename)
dimage

###Determine image dimensions

In [0]:
train_data_dir = '/content/drive/My Drive/Colab Notebooks/cars_dl/cars/training_set/'
validation_data_dir = '/content/drive/My Drive/Colab Notebooks/cars_dl/cars/val_set/'
test_data_dir = '/content/drive/My Drive/Colab Notebooks/cars_dl/cars/test_set/'
nb_train_samples = trainingCarImages + trainingCrashImages
nb_validation_samples = validCarImages + validCrashImages
nb_test_samples = testCarImages + testCrashImages

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

print(input_shape)

In [0]:
#Method to print several images in a single row 
def plots(img, figsize=(12,6), rows = 1, titles = 1):
    if type(img[0]) is np.ndarray:
        img = np.array(img).astype(np.float_)
        if (img.shape[-1] != 3):
            img = img.transpose((0, 2, 3, 1))
    f =plt.figure(figsize = figsize)
    cols = 7//rows if (len(img) % 2 == 0) else len(img)//rows + 1
    for i in range(cols):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize = 10)
        plt.imshow(img[i], interpolation = None if np.interp else 'none')

##Create CNN model

In [0]:
# Initialising the CNN
classifier = Sequential()

# Layer 1 - convolution
classifier.add(Convolution2D(32, (3, 3), 
              input_shape = (img_width, img_height, 3), 
              activation = 'relu'))                        # Convolution
classifier.add(MaxPooling2D(pool_size = (2, 2)))          # MaxPooling
classifier.add(Dropout(0.25))                             # Dropout


#Add another convolutional layer
# classifier.add(Convolution2D(32, 3, 3, 
#               activation = 'relu'))
# classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Layer 2 - convolution
classifier.add(Convolution2D(32, (3, 3), 
              activation = 'relu', 
              kernel_regularizer=regularizers.l2(0.01)))  # Convolution layer
classifier.add(MaxPooling2D(pool_size = (2, 2)))          # MaxPooling
classifier.add(Dropout(0.25))                             # Dropout

# Layer 3 - convolution
classifier.add(Convolution2D(32, (3, 3), 
              activation = 'relu'))                       # Convolution layer
classifier.add(MaxPooling2D(pool_size = (2, 2)))          # MaxPooling
classifier.add(Dropout(0.25))                             # Dropout

# Layer 4 - convolution
classifier.add(Convolution2D(32, (3, 3), 
              activation = 'relu'))                       # Convolution layer
classifier.add(MaxPooling2D(pool_size = (2, 2)))          # MaxPooling
classifier.add(Dropout(0.25))                             # Dropout

classifier.add(Flatten())                                 # Flattening

# Fully connected layer
classifier.add(Dense(output_dim = 128, 
                     activation = 'relu'))                # Dense layer
classifier.add(Dropout(0.25))                             # Dropout
classifier.add(Dense(output_dim = 2, 
                     activation = 'softmax'))             # Output

classifier.summary()

##Compiling the CNN

In [0]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

##Data standardization and augmentation of training data

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True
                                   )

training_set = train_datagen.flow_from_directory(train_data_dir,
                                                 target_size = (img_width, img_height),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=42)

test_datagen = ImageDataGenerator(rescale = 1./255)

val_set = test_datagen.flow_from_directory(validation_data_dir,
                                            target_size = (img_width, img_height),
                                            batch_size = batch_size,
                                            class_mode = 'categorical',
                                            shuffle=True,
                                            seed=42)

test_set = test_datagen.flow_from_directory(test_data_dir,
                                            target_size = (img_width, img_height),
                                            batch_size = 1,
                                            class_mode = 'categorical',
                                            shuffle=True,
                                            seed=42)


##Print normalized and re-sized images from training set

In [0]:
imgs, labels = next(training_set)
plots(imgs, titles = labels)

##Print normalized and re-sized images from validation set

In [0]:
imgs, labels = next(val_set)
plots(imgs, titles = labels)

In [0]:
#Method to identify and remove corrupted images
def my_gen(gen):
    while True:
        try:
            imgs, labels = next(gen)
            yield imgs, labels
        except:
            pass

##Training the CNN model

In [0]:
history = classifier.fit_generator(my_gen(training_set),
                         steps_per_epoch = 78,
                         epochs = epochs,
                         validation_data = val_set,
                         validation_steps = validation_steps)

In [0]:
# list all data in history
print(history.history.keys())

## Save neural network structure

In [0]:
model_structure = classifier.to_json()
f = Path("/content/drive/My Drive/Colab Notebooks/model_structure.json")
f.write_text(model_structure)

## Save neural network's trained weights

In [0]:
# Save neural network's trained weights
classifier.save_weights("/content/drive/My Drive/Colab Notebooks/model_weights.h5")

##Plot training and validation accuracy per epoch

In [0]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

##Plot training and validation loss per epoch

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
training_set.class_indices

##Prediction

In [0]:
testCarImages = os.listdir('/content/drive/My Drive/Colab Notebooks/cars_dl/cars/test_set/')
testFilename = '/content/drive/My Drive/Colab Notebooks/cars_dl/cars/test_set/' + testCarImages[20]

test_image = image.load_img(testFilename, target_size = (img_width, img_height))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = classifier.predict(test_image)
test_set.class_indices
if result[0][0] < 0.3:
  prediction = 'No'
elif result[0][0] > 0.7: 
  prediction = 'Yes'
else:
  prediction = 'Unsure'


dimage = Image.open(testFilename)
dimage


In [0]:
print(prediction + ": " + str(result[0][0]))

##Confution Matrix and Classification Report

In [0]:
Y_pred = classifier.predict_generator(test_set, len(validCrashImages)*2 // batch_size + 1)

y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')

print(confusion_matrix(test_set.classes, y_pred))

print('Classification Report')

target_names = ['Yes', 'Unsure', 'No']

print(classification_report(test_set.classes, y_pred, target_names=target_names))